# Imports

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel
import torch
import os
import re
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import spacy
from nltk.stem import WordNetLemmatizer
import requests
import time
from transformers import pipeline
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MarianMTModel, MarianTokenizer


# File loading

In [2]:
# Load the files 
question_globales_path = 'data/QuestionGlobales.xlsx'
extraction_codebook_path = 'data/Extraction CodeBook - 3. Cleaned.xlsx'
gsb_path = 'data/GSB 2023_V1.xlsx'

# Read the global questions file
df_globales = pd.read_excel(question_globales_path)

# Read all sheets from the codebook file
sheets_codebook = pd.read_excel(extraction_codebook_path, sheet_name=None)

# answers dataframe
gsb_df = pd.read_excel(gsb_path) 


In [3]:
gsb_df.head()

,BFS_2023,Gemeinde_2023,GSB23_Teilnahme,GSB23_Mode,GSB23_StartDate,GSB23_EndDate,GSB23_Progress,GSB23_Duration__in_seconds_,GSB23_Finished,GSB23_RecordedDate,...,gexesvpat,gexespsat,gexeandat,gexeploat,stfussv,orgpart,autonomie,zamitkt,wahlverf,legislative
0,1,Aeugst am Albis,1,0.0,2023-02-16 08:34:08,2023-02-16 10:16:09,100.0,6121.0,1.0,2023-02-16 10:16:10,...,0.000000,0.000000,66.666667,0.000000,1.0,1.0,7.0,3.0,1.0,1.0
1,2,Affoltern am Albis,1,0.0,2023-02-16 08:50:23,2023-02-17 10:17:44,100.0,91641.0,1.0,2023-02-17 10:17:45,...,12.500000,12.500000,50.000000,12.500000,2.0,1.0,5.0,3.0,1.0,1.0
2,3,Bonstetten,1,0.0,2023-03-30 10:16:48,2023-03-30 11:49:16,100.0,5547.0,1.0,2023-03-30 11:49:17,...,16.666667,0.000000,0.000000,66.666667,2.0,1.0,5.0,2.0,1.0,1.0
3,4,Hausen am Albis,1,0.0,2023-02-28 10:27:13,2023-02-28 11:45:36,100.0,4702.0,1.0,2023-02-28 11:45:37,...,14.285714,14.285714,0.000000,42.857143,2.0,1.0,6.0,2.0,1.0,1.0
4,5,Hedingen,1,0.0,2023-02-20 14:13:19,2023-02-20 14:34:49,40.0,1289.0,0.0,NaT,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN


## Start of data manipulation before NLP preprocessing pipeline

In [4]:
# create a dataframe for each year
years = ['1988', '1994', '1998', '2005', '2009', '2017', '2023']
dataframes = {}

# choose the columns to keep
columns_to_keep = ['code', 'num_question', 'year', 'label', 'type', 'format']


for year in years:
    if year in sheets_codebook:  # ensure the sheet exists
        # each sheet is a DataFrame
        df = sheets_codebook[year]
        # keep only the columns we need (defined in columns_to_keep)
        dataframes[f'main_questions_{year}'] = df[columns_to_keep]
    else:
        print(f"Sheet for year {year} not found.")

# define the dataframes
main_questions_1988 = dataframes.get('main_questions_1988', pd.DataFrame())
main_questions_1994 = dataframes.get('main_questions_1994', pd.DataFrame())
main_questions_1998 = dataframes.get('main_questions_1998', pd.DataFrame())
main_questions_2005 = dataframes.get('main_questions_2005', pd.DataFrame())
main_questions_2009 = dataframes.get('main_questions_2009', pd.DataFrame())
main_questions_2017 = dataframes.get('main_questions_2017', pd.DataFrame())
main_questions_2023 = dataframes.get('main_questions_2023', pd.DataFrame())

main_questions_2023.head()

,code,num_question,year,label,type,format
0,GSB23_Teilnah- me,Teilnah- me,2023,Hat die Gemeinde an der Gemeindeschreiberbefra...,discrete,numeric-40.0
1,GSB23_Mode,Mode,2023,Teilnahme-Modus,discrete,numeric-40.0
2,GSB23_StartDate,StartDate,2023,Start Date,discrete,character-20
3,GSB23_EndDate,EndDate,2023,End Date,discrete,character-20
4,GSB23_Progress,Progress,2023,Progress,continuous,numeric-40.2


## Sheet filtering for num_question

In [5]:
# tailor made data cleaning for the old years sheets 
def filter_num_question_old(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  

    # convert num_question to string
    df.loc[:, 'num_question'] = df['num_question'].astype(str)  

    # some tailor made filtering --> keep only rows with num_question that are digits or contain 'a' or 'A' 
    df = df[df['num_question'].str.match(r'^\d+$|.*[aA].*')]
    
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    # delete the a or A from the num_question
    df['num_question'] = df['num_question'].str.replace('a', '', regex=False).str.replace('A', '', regex=False)

    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    return df

In [6]:
# tailor made data cleaning for the new years sheets 
def filter_num_question_new(df):
    # delete rows with empty num_question
    df = df[df['num_question'].notna()].copy()  
    # keep only the first occurence of each num_question
    df = df.drop_duplicates(subset='num_question', keep='first')
    return df

In [7]:
main_questions_1988 = filter_num_question_old(main_questions_1988)
main_questions_1994 = filter_num_question_old(main_questions_1994)
main_questions_1998 = filter_num_question_old(main_questions_1998)
main_questions_2005 = filter_num_question_old(main_questions_2005)
main_questions_2009 = filter_num_question_old(main_questions_2009)
main_questions_2017 = filter_num_question_new(main_questions_2017)
main_questions_2017 = main_questions_2017[main_questions_2017['num_question'] != 'Q1']
main_questions_2023 = filter_num_question_new(main_questions_2023)


In [8]:
main_questions_2017.head()

,code,num_question,year,label,type,format
0,GSB17_Teilnahme,Teilnahme,2017,Hat die Gemeinde an der Gemeindeschreiber...,NaN,NaN
1,GSB17_1,1,2017,Gemeinden können in unterschiedlichem Masse vo...,NaN,NaN
16,GSB17_Q2,Q2,2017,"Es kann Probleme geben, welche die ...",NaN,NaN
49,GSB17_Q3,Q3,2017,Wie hat sich der Steuerfuss 2017 Ihrer Gemeind...,NaN,NaN
50,GSB17_Q4,Q4,2017,Wie hat sich der reale Ertrag aus ...,NaN,NaN


In [9]:
dataframes_to_merge = [
    main_questions_1988,
    main_questions_1994,
    main_questions_1998,
    main_questions_2005,
    main_questions_2009,
    main_questions_2017,
    main_questions_2023
]

# merge all the dataframes
merged_df = pd.concat(dataframes_to_merge, ignore_index=True)

In [10]:
merged_df.head()

,code,num_question,year,label,type,format
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN


## Start of text preprocessing

In [11]:
!python -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     ---------------------------------------- 0.1/14.6 MB 1.2 MB/s eta 0:00:13
      --------------------------------------- 0.2/14.6 MB 2.3 MB/s eta 0:00:07
     - -------------------------------------- 0.4/14.6 MB 2.3 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/14.6 MB 3.2 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/14.6 MB 4.3 MB/s eta 0:00:04
     ---- ----------------------------------- 1.7/14.6 MB 4.9 MB/s eta 0:00:03
     ----- ---------------------------------- 1.9/14.6 MB 4.9 MB/s eta 0:00:03
     ------ --------------------------------- 2.3/14.6 MB 5.2 MB/s eta 0:00:03
     ------- -------------------------------- 2.7/14.6 MB 5.5 MB/s eta 0:00:03
     -------- ------------------------------- 3.0/14.6 MB 5.6 MB/s

In [12]:
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # 1. convert text to lowercase
    text = text.lower()
    
    # 2. delete punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # 3. delete extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. lemmatization + tokenization + remove stopwords
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop] 
    
    return tokens  

# apply the preprocess_text function to the 'label' column
tqdm.pandas() 
merged_df['tokens'] = merged_df['label'].progress_apply(preprocess_text)


100%|██████████| 563/563 [00:02<00:00, 225.13it/s]


In [13]:
merged_df.head()

,code,num_question,year,label,type,format,tokens
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN,"[Postleitzahl, Gemeinde]"
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN,"[wieviele, Einwohner, umfassen, Gemeinde]"
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN,"[Beschreibung, treffen, Gemeinde]"
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN,"[Bezeichnung, Treffen, eher, Gemeinde]"
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN,"[zusammenleben, Gemeinde, betrachten, extrem, ..."


In [14]:
merged_df[merged_df['year'] == 2023]

,code,num_question,year,label,type,format,tokens
481,GSB23_Teilnah- me,Teilnah- me,2023,Hat die Gemeinde an der Gemeindeschreiberbefra...,discrete,numeric-40.0,"[Gemeinde, Gemeindeschreiberbefragung, 2023, t..."
482,GSB23_Mode,Mode,2023,Teilnahme-Modus,discrete,numeric-40.0,[Teilnahmemodus]
483,GSB23_StartDate,StartDate,2023,Start Date,discrete,character-20,"[Start, Date]"
484,GSB23_EndDate,EndDate,2023,End Date,discrete,character-20,"[End, dat]"
485,GSB23_Progress,Progress,2023,Progress,continuous,numeric-40.2,[progress]
...,...,...,...,...,...,...,...
558,GSB23_Q61,Q61,2023,"61. Sind Sie in der Gemeinde, in der Sie arbei...",continuous,numeric-40.0,"[61, Gemeinde, arbeiten, Wohnhaft]"
559,GSB23_Q62,Q62,2023,62. Sind Sie auch in anderen Gemeinden beschäf...,continuous,numeric-40.0,"[62, Gemeinde, beschäftigen]"
560,GSB23_Q63,Q63,2023,63. Wurden Sie gewählt\noder angestellt?,continuous,numeric-40.0,"[63, wählen, anstellen]"
561,GSB23_Q64,Q64,2023,64. Stehen Sie einer bestimmten Partei nahe?,continuous,numeric-40.0,"[64, stehen, bestimmt, Partei, nahe]"


# NLP pipeline: 

## Transformer version: Bert

In [15]:
model_name = 'dbmdz/bert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [16]:
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc='Generating embeddings'):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(embedding)
    return np.vstack(embeddings)


In [17]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()  # convert the list of tokens to a string

# obtain the embeddings for the tokens
embeddings = get_embeddings(labels)

# use cosine similarity to calculate the similarity matrix
similarity_matrix = cosine_similarity(embeddings)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# set the threshold for similarity --> here 0.97 but can be changed
threshold = 0.97

similar_pairs = []

# iterate over the similarity matrix to identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):  
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  # ad id for each combined row
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 1967.64it/s]


In [18]:
question_globale_new.head()

,id,label,year,code
0,1,wieviele administrativ tätig angestellter Beam...,1988; 1994,GSB88_8; GSB94_39a
1,2,stark persönlich Einschätzung allgemein einflu...,1988; 2005; 2009,GSB88_42; GSB05_30; GSB09_29
2,3,steuerfuss Steueranlage Gemeinde Vergleich 199...,1994; 1998; 2005,GSB94_15; GSB98_11; GSB05_3
3,4,steuerfuss Steueranlage Gemeinde Vergleich 199...,1994; 1998; 2009,GSB94_15; GSB98_11; GSB09_2
4,5,real Ertrag Einkommen Vermögenssteuer Gemeinde...,1994; 1998; 2005,GSB94_16; GSB98_12; GSB05_4


In [19]:
question_globale_new.shape

(43, 4)

In [20]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei; wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen respektive 10 beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei'

## Transformer version: DSBSAN

In [21]:
# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# embedding model
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc="embedding computation", unit="label"):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()  
            embeddings.append(embedding)
    return np.vstack(embeddings)

# obtain unique labels
labels = merged_df['label'].unique()

# obtain embeddings for the labels
embeddings = get_embeddings(labels)
print("Shape of embeddings:", embeddings.shape)

# use cosine similarity to calculate the similarity matrix
distance_matrix = cosine_distances(embeddings)

# apply DBSCAN clustering
dbscan = DBSCAN(eps=0.05, min_samples=3, metric='precomputed')  # use precomputed to pass the distance matrix
clusters = dbscan.fit_predict(distance_matrix)

# create a dictionary to map labels to indices
label_to_index = {label: index for index, label in enumerate(merged_df['label'].unique())}

# create a Series to store the clusters
cluster_series = pd.Series(-1, index=merged_df.index)

# assign the clusters to the labels
for label, cluster_id in zip(labels, clusters):
    index = label_to_index[label]
    cluster_series.iloc[index] = cluster_id

# add the clusters to the merged DataFrame
merged_df['cluster'] = cluster_series

# create a new DataFrame to store the combined rows
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

# fill the new DataFrame with the combined rows
id_counter = 1
for cluster_id in set(merged_df['cluster']):
    if cluster_id != -1:  
        temp_df = merged_df[merged_df['cluster'] == cluster_id]
        if not temp_df.empty:
            combined_row = {
                'id': id_counter,
                'label': '; '.join(temp_df['label']),
                'year': '; '.join(temp_df['year'].astype(str)),
                'code': '; '.join(temp_df['code'])
            }
            question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)
            id_counter += 1


embedding computation: 100%|██████████| 549/549 [00:22<00:00, 24.49label/s]

Shape of embeddings: (549, 768)


In [22]:
question_globale_new.shape

(18, 4)

In [23]:
question_globale_new.head()

,id,label,year,code
0,1,Wenn in Ihrer Gemeinde das Amt des Schreibers ...,1988; 1988; 1988; 1988; 1988; 1988; 1988; 1988...,GSB88_9; GSB88_11; GSB88_21a; GSB88_22a; GSB88...
1,2,Wurde von einer Partei bei den letzten allgeme...,1988; 1988; 2005; 2009; 2009; 2017,GSB88_27a; GSB88_28a; GSB05_41a1; GSB09_37; GS...
2,3,Wurden in den letzten 5 Jahren Entscheid...,1988; 1994; 2005,GSB88_38; GSB94_30a; GSB05_22a
3,4,Nach welchem Wahlverfahren wird das Parlament ...,1988; 1998; 2017; 2017; 2017,GSB88_39; GSB98_40; GSB17_Q27; GSB17_Q135_2; G...
4,5,Wie würden Sie die Zusammenarbeit zw...,1988; 1988; 1994; 1994; 1994; 1994; 1998; 1998...,GSB88_47; GSB88_57; GSB94_13a16; GSB94_14a; GS...


In [24]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'Wenn in Ihrer Gemeinde das Amt des Schreibers (oder eines anderen Chefbeamten)    neu    besetzt    werden    soll:    wird    da    auf    die\nParteimitgliedschaft der Bewerber geachtet?; Ist   der   Präsident   (LU:   Ammann)   Ihrer   Gemeinde   vollamtlich, halbamtlich oder nebenamtlich tätig?; Sind  Selbständigerwerbende  im Gemeindevorstand  (inkl.  Präsident)\nvertreten?; Befinden sich unter den Vorstandsmitgliedern (inkl. Präsident) auch Mandatsträger auf kantonaler oder nationaler Ebene?; Wieviele Kandidaten haben sich insgesamt bei den letzten ordent- lichen  Wahlen  um  Sitze  im  Gemeindevorstand  beworben,  und  von\nwelcher Partei (Partei und Anzahl) wurden sie gestellt?; Wie oft kommt es vor, dass Vorstandsmitglieder, die derselben Partei angehören,    bei    Sachentscheiden    gegensätzliche    Auffassungen\nvertreten?; Gab   es   bei   den   letzten   oder   vorletzten   allgemeinen   Wahlen amtierende Vorstandsmitglieder, die erneut kandidierten, aber nicht\nwiederg

## TF-IDF version

### Top 10 global questions

In [25]:
def has_duplicates(years):
    return len(years) != len(set(years))

# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

similarity_matrix[np.tril_indices_from(similarity_matrix)] = -1

top_10_indices = np.dstack(np.unravel_index(np.argsort(similarity_matrix.ravel())[-10:], similarity_matrix.shape))[0]

question_globale_new_top_10 = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

# Traiter les 10 paires ayant la plus grande similarité
for i, j in top_10_indices:
    label1 = labels[i]
    label2 = labels[j]
    
    # Ajouter une ligne pour chaque paire de tokens similaires
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    
    combined_years = '; '.join(temp_df['year'].astype(str)).split('; ')

    # Vérifier s'il y a des années dupliquées
    if has_duplicates(combined_years):
        continue  # Ignorer la ligne si des années dupliquées sont trouvées
    
    # Construire la ligne combinée
    combined_row = {
        'id': id_counter,  
        'label': f"{label1}; {label2}",
        'year': '; '.join(combined_years),
        'code': '; '.join(temp_df['code'])
    }

    # Ajouter la ligne au DataFrame question_globale_new
    question_globale_new_top_10 = pd.concat([question_globale_new_top_10, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


In [26]:
question_globale_new_top_10.shape

(9, 4)

In [27]:
question_globale_new_top_10.head(50)

,id,label,year,code
0,1,arbeiten derder Gemeindepräsidentin aktiv Verw...,2017; 2023,GSB17_Q58; GSB23_Q52
1,2,wieviele Mitglied Gemeindeexekutive Inkl Präsi...,1994; 2005; 2009,GSB94_27; GSB05_38; GSB09_34
2,3,Polarisierung lokal Parteienlandschaft letzter...,1994; 2005,GSB94_25; GSB05_33
3,4,letzter 10 leicht schwierig Vakant Ämter Gemei...,1994; 2005,GSB94_38a; GSB05_37
4,5,politisch Auseinandersetzung Gemeinde Auge hal...,1988; 1994; 2005,GSB88_52; GSB94_21; GSB05_29
5,6,Gemeinde mitwirkungsmöglichkeiten Einwohnersch...,1994; 2005,GSB94_36; GSB05_50
6,7,folgend politisch Partei politisch Gruppierung...,1994; 2005,GSB94_24; GSB05_35
7,8,Anteil Steuereinnahme Gesamteinnahm Gemeinde; ...,2017; 2023,GSB17_Q7; GSB23_Q9
8,9,stehen Gemeinde kantonal Finanzausgleich; 7 st...,2017; 2023,GSB17_Q6; GSB23_Q7


In [28]:
question_globale_new_top_10.loc[question_globale_new_top_10['year'].str.contains('2023'), 'label']


0    arbeiten derder Gemeindepräsidentin aktiv Verw...
7    Anteil Steuereinnahme Gesamteinnahm Gemeinde; ...
8    stehen Gemeinde kantonal Finanzausgleich; 7 st...
Name: label, dtype: object

In [29]:
question_globale_new_top_10[question_globale_new_top_10['id'] == 1].label.values[0]

'arbeiten derder Gemeindepräsidentin aktiv Verwaltung dh erledigen Ersie administrativ Tätigkeit ähnlich verwaltungsmitarbeitend; 52 arbeiten derder Gemeindepräsidentin aktiv Verwaltung dh erledigen Ersie administrativ Tätigkeit ähnlich verwaltungsmitarbeitend'

### Question global selection based on threshold 

In [30]:
def has_duplicates(years):
    return len(years) != len(set(years))

# obtain the tokens from the merged DataFrame
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

# create a DataFrame from the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# define the threshold for similarity, here 0.8 seems to be a good value but can be changed 
threshold = 0.6

similar_pairs = []

# identify similar pairs
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# create a new DataFrame to store the combined rows
question_globale_new_threshold = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # add a row for each pair of similar tokens
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    
    combined_years = '; '.join(temp_df['year'].astype(str)).split('; ')

    if has_duplicates(combined_years):
        continue
    
    combined_row = {
        'id': id_counter,  
        'label': f"{label1}; {label2}",
        'year': '; '.join(combined_years),
        'code': '; '.join(temp_df['code'])
    }

    question_globale_new_threshold = pd.concat([question_globale_new_threshold, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1

Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 14308.76it/s]


In [31]:
question_globale_new_threshold.shape

(148, 4)

### Here the results seems better --> use the TF-IDF version 

# CSV extraction of the global questions

In [32]:
quest_glob_columns = [
    'label', 'code_first_question','code_other_question','text_de', 'text_fr', 'text_it', 'text_ro', 'text_en',
    'category_label', 'category_text_de', 'category_text_fr',
    'category_text_it', 'category_text_ro', 'category_text_en',
    'options_value', 'options_label'
]

code_to_token = {}

for index, row in merged_df.iterrows():
    code = row['code']
    tokens = row['tokens']

    for token in tokens:
        if not token.isdigit():  
            if code not in code_to_token:  
                code_to_token[code] = token
            break


print(code_to_token)

{'GSB88_1a': 'Postleitzahl', 'GSB88_2a': 'wieviele', 'GSB88_3': 'Beschreibung', 'GSB88_4': 'Bezeichnung', 'GSB88_5a': 'zusammenleben', 'GSB88_6': 'Gemeinde', 'GSB88_7': 'wieviele', 'GSB88_8': 'wieviele', 'GSB88_9': 'Gemeinde', 'GSB88_10a': 'Gemeindeschreiber', 'GSB88_11': 'Präsident', 'GSB88_12a': 'Präsident', 'GSB88_13': 'Partei', 'GSB88_14a': 'Kandidat', 'GSB88_15': 'Ort', 'GSB88_16': 'wahlverfahren', 'GSB88_17a': 'Anzahl', 'GSB88_19a': 'Frau', 'GSB88_20a': 'finden', 'GSB88_21a': 'Selbständigerwerbend', 'GSB88_22a': 'befinden', 'GSB88_23a': 'wieviele', 'GSB88_24': 'möchten', 'GSB88_25': 'vorstandsmitglied', 'GSB88_26a': 'letzter', 'GSB88_27a': 'Partei', 'GSB88_28a': 'Partei', 'GSB88_29': 'letzter', 'GSB88_30': 'Gemeinde', 'GSB88_31a': 'wieviele', 'GSB88_32': 'häufig', 'GSB88_33a': 'bestehen', 'GSB88_34': 'Gemeinde', 'GSB88_35': 'Gemeinde', 'GSB88_36a': 'wievieler', 'GSB88_37': 'haufig', 'GSB88_38': 'letzter', 'GSB88_39': 'wahlverfahren', 'GSB88_40a': 'festgelegt', 'GSB88_41': 'Gemein

In [41]:
question_globale_new_top_10.head()

,id,label,year,code
0,1,arbeiten derder Gemeindepräsidentin aktiv Verw...,2017; 2023,GSB17_Q58; GSB23_Q52
1,2,wieviele Mitglied Gemeindeexekutive Inkl Präsi...,1994; 2005; 2009,GSB94_27; GSB05_38; GSB09_34
2,3,Polarisierung lokal Parteienlandschaft letzter...,1994; 2005,GSB94_25; GSB05_33
3,4,letzter 10 leicht schwierig Vakant Ämter Gemei...,1994; 2005,GSB94_38a; GSB05_37
4,5,politisch Auseinandersetzung Gemeinde Auge hal...,1988; 1994; 2005,GSB88_52; GSB94_21; GSB05_29


In [33]:
# create a new DataFrame to store the final results
df_fin_top_10 = pd.DataFrame(columns=quest_glob_columns)

# fill the new DataFrame with the combined rows
for index, row in question_globale_new_top_10.iterrows():
    codes_list = row['code'].split('; ')
    gsb23_code = next((code for code in row['code'].split('; ') if code.startswith('GSB23_')), None)
    code_first_question = gsb23_code if gsb23_code is not None else row['code'].split('; ')[0]  # use 'GSB23_' code or the first code
    code_other_question = '; '.join([code for code in codes_list if code != code_first_question])
    first_token = code_to_token.get(code_first_question, '')  # use the token corresponding to the first code

    # Find the corresponding text_de in merged_df using code_first_question
    label_from_merged = merged_df.loc[merged_df['code'] == code_first_question, 'label'].values
    if len(label_from_merged) > 0:
        # Remove any leading numbers and dots (e.g., "52. " or "123. ")
        cleaned_label = re.sub(r'^\d+\.\s*', '', label_from_merged[0])
        text_de_first_question = cleaned_label
    else:
        text_de_first_question = row['label'].split('; ')[0]

    # create a new row with the required columns
    new_row = {
        'label': first_token,  # use the first code in 'label'
        'code_first_question': code_first_question,  # put the first code in 'code_first_question'
        'code_other_question': code_other_question,  # put the other codes in 'code_other_question'
        'text_de': text_de_first_question,  # use the first label in 'text_de'
        'text_fr': '',  
        'text_it': '', 
        'text_ro': '', 
        'text_en': '',
        'category_label': '',
        'category_text_de': '',
        'category_text_fr': '',
        'category_text_it'  : '',
        'category_text_ro': '',
        'category_text_en': '',
        'options_value': '',
        'options_label': ''
    }

    # add the new row to the final DataFrame
    df_fin_top_10 = pd.concat([df_fin_top_10, pd.DataFrame([new_row])], ignore_index=True)


df_fin_top_10.head()


,label,code_first_question,code_other_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,arbeiten,GSB23_Q52,NaN,Arbeitet der/die Gemeindepräsident/-in aktiv i...,,,,,,,,,,,,
1,wieviele,GSB94_27,NaN,Wieviele Mitglieder der Gemeindeexeku...,,,,,,,,,,,,
2,Polarisierung,GSB94_25,NaN,Hat die Polarisierung in der lokalen Par...,,,,,,,,,,,,
3,letzter,GSB94_38a,NaN,Ist es in den letzten 10 Jahren leichte...,,,,,,,,,,,,
4,politisch,GSB88_52,NaN,Wenn Sie sich die politischen Ausein...,,,,,,,,,,,,


In [34]:
gsb_columns = gsb_df.columns

# fill the 'options_value' column in df_fin
for index, row in df_fin_top_10.iterrows():
    # get the code of the first question
    question_code = row['code_first_question']
    
    # check if the question code exists in the 'GSB 2023_V1' DataFrame
    if question_code in gsb_columns:
        # extract the unique values from the column
        unique_values = gsb_df[question_code].dropna().unique()
        
        # if there are unique values, join them 
        if len(unique_values) > 0:
            options_value = ";".join(map(str, unique_values))
        else:
            options_value = ''
        
        # fill the options_value in the final DataFrame
        df_fin_top_10.at[index, 'options_value'] = options_value
    else:
        # if the question code does not exist in the 'GSB 2023_V1' DataFrame, fill an empty string
        # to be adapted when other version of the answers than 2023 will be available
        df_fin_top_10.at[index, 'options_value'] = ''


In [35]:
df_fin_top_10.head(10)

,label,code_first_question,code_other_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,arbeiten,GSB23_Q52,NaN,Arbeitet der/die Gemeindepräsident/-in aktiv i...,,,,,,,,,,,2.0;1.0;-99.0,
1,wieviele,GSB94_27,NaN,Wieviele Mitglieder der Gemeindeexeku...,,,,,,,,,,,,
2,Polarisierung,GSB94_25,NaN,Hat die Polarisierung in der lokalen Par...,,,,,,,,,,,,
3,letzter,GSB94_38a,NaN,Ist es in den letzten 10 Jahren leichte...,,,,,,,,,,,,
4,politisch,GSB88_52,NaN,Wenn Sie sich die politischen Ausein...,,,,,,,,,,,,
5,Gemeinde,GSB94_36,NaN,Für alle Gemeinden: Sind d...,,,,,,,,,,,,
6,folgend,GSB94_24,NaN,Welche der folgenden politische...,,,,,,,,,,,,
7,Anteil,GSB23_Q9,NaN,Wie gross ist der Anteil der Steuereinnahmen a...,,,,,,,,,,,3.0;99.0;5.0;2.0;1.0;4.0;-99.0,
8,stehen,GSB23_Q7,NaN,Wie steht Ihre Gemeinde im kantonalen Finanzau...,,,,,,,,,,,1.0;2.0;99.0;4.0;3.0;-99.0,


## Translation of the global question using an open source API


### Libretranslate -> not working locally 

### Bert model to translate 

In [36]:
# model selection, also tried "Helsinki-NLP/opus-mt-de-fr" but the translation was not as good
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

def translate_mbart(text, source_lang, target_lang):
    # source language
    tokenizer.src_lang = source_lang
    # encode
    encoded_input = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
        max_length=512,
        num_beams=4,  # use beam search to improve the translation
        early_stopping=True
    )
    # decode the generated tokens
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]



In [37]:
df_fin_top_10['text_en'] = df_fin_top_10['text_de'].apply(lambda x: translate_mbart(x, "de_DE", "en_XX"))
df_fin_top_10['text_fr'] = df_fin_top_10['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "fr_XX"))
df_fin_top_10['text_it'] = df_fin_top_10['text_en'].apply(lambda x: translate_mbart(x, "en_XX", "it_IT"))
# romanche not available in the model

In [38]:
df_fin_top_10['text_ro'] = 'Not available for the moment'

In [39]:
df_fin_top_10.head(10)

,label,code_first_question,code_other_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,arbeiten,GSB23_Q52,NaN,Arbeitet der/die Gemeindepräsident/-in aktiv i...,Le maire participe-t-il activement à l'adminis...,Il sindaco partecipa attivamente all'amministr...,Not available for the moment,Does the mayor actively participate in the adm...,,,,,,,2.0;1.0;-99.0,
1,wieviele,GSB94_27,NaN,Wieviele Mitglieder der Gemeindeexeku...,Combien de membres de la direction municipale ...,Quanti membri dell'esecutivo municipale (inclu...,Not available for the moment,How many members of the municipal executive (i...,,,,,,,,
2,Polarisierung,GSB94_25,NaN,Hat die Polarisierung in der lokalen Par...,La polarisation dans le paysage des partis loc...,La polarizzazione nel panorama locale dei part...,Not available for the moment,Has polarization in the local party landscape ...,,,,,,,,
3,letzter,GSB94_38a,NaN,Ist es in den letzten 10 Jahren leichte...,"Au cours des 10 dernières années, a-t-il été p...","Negli ultimi 10 anni, è diventato più facile o...",Not available for the moment,"In the last 10 years, has it become easier or ...",,,,,,,,
4,politisch,GSB88_52,NaN,Wenn Sie sich die politischen Ausein...,Si vous avez à l'esprit les conflits politique...,Se tenete a mente i conflitti politici nella v...,Not available for the moment,If you keep in mind the political conflicts in...,,,,,,,,
5,Gemeinde,GSB94_36,NaN,Für alle Gemeinden: Sind d...,Pour toutes les municipalités: Les possibilité...,Per tutte le municipalità: Le possibilità di p...,Not available for the moment,For all municipalities: Have the possibilities...,,,,,,,,
6,folgend,GSB94_24,NaN,Welche der folgenden politische...,Quels des partis et groupes politiques* suivan...,Quali dei partiti e dei gruppi politici* che p...,Not available for the moment,Which of the following political parties and p...,,,,,,,,
7,Anteil,GSB23_Q9,NaN,Wie gross ist der Anteil der Steuereinnahmen a...,Quelle est la part des recettes fiscales dans ...,Quanto è grande la percentuale dei ricavi fisc...,Not available for the moment,How large is the share of tax revenues in the ...,,,,,,,3.0;99.0;5.0;2.0;1.0;4.0;-99.0,
8,stehen,GSB23_Q7,NaN,Wie steht Ihre Gemeinde im kantonalen Finanzau...,Quelle est la position de votre municipalité e...,Come si presenta la vostra municipalità nella ...,Not available for the moment,How does your municipality stand in cantonal f...,,,,,,,1.0;2.0;99.0;4.0;3.0;-99.0,


### Results extraction

In [40]:
df_fin_top_10.to_csv('data/top_10_QuestionGlobales_NLP.csv', index=False, encoding='utf-8')